## Lab 6.3 - Improving the model

In this section of the lab, you will be asked to apply what you have learned to create a RNN model that can generate new sequences of text based on what it has learned from a large set of existing text. In this case we will be using the full text of Lewis Carroll's *Alice in Wonderland*. Your task for the assignment is to:

- format the book text into a set of training data
- define a RNN model in Keras based on one or more LSTM or GRU layers
- train the model with the training data
- use the trained model to generate new text

Our previous model based on Obama's essay was prone to overfitting since there was not that much data to learn from. Thus, the generated text was either unintelligeable (not enough learning) or exactly replicated the training data (over-fitting). In this case, we are working with a much bigger data set, which should provide enough data to avoid over-fitting, but will also take more time to train. To improve your model, you can experiment with tuning the following hyper-parameters:

- Use more than one recurrent layer and/or add more memory units (hidden neurons) to each layer. This will allow you to learn more complex structures in the data.
- Use sequences longer than 100 characters, which will allow you to learn from patterns further back in time.
- Change the way the sequences are generated. For example you could try to break up the text into real sentances using the periods, and then either cut or pad each sentance to make it 100 characters long.
- Increase the number of training epochs, which will give the model more time to learn. Monitor the validation loss at each epoch to make sure the model is still improving at each epoch and is not overfitting the training data.
- Add more dropout to the recurrent layers to minimize over-fitting.
- Tune the batch size - try a batch size of 1 as a (very slow) baseline and larger sizes from there.
- Experiment with scale factors (temperature) when interpreting the prediction probabilities.

If you get an error such as `alloc error` or `out of memory error` during training it means that  your computer does not have enough RAM memory to store the model parameters or the batch of training data needed during a training step. If you run into this issue, try reducing the complexity of your model (both number and depth of layers) or the mini-batch size.

The last three code blocks will use your trained model to generate a sequence of text based on a predefined seed. Do not change any of the code, but run it before submitting your assignment. Your work will be evaluated based on the quality of the generated text. A good result should be legible with decent grammar and spelling (this indicates a high level of learning), but the exact text should not be found anywhere in the actual text (this indicates over-fitting).

Let's start by importing the libraries we will be using, and importing the full text from Alice in Wonderland:

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from time import gmtime, strftime
import os
import re
import pickle
import random
import sys

Using Theano backend.


In [2]:
filename = "data/wonderland.txt"
raw_text = open(filename).read()

raw_text = re.sub('[^\nA-Za-z0-9 ,.:;?!-]+', '', raw_text)
raw_text = raw_text.lower()

n_chars = len(raw_text)
print "length of text:", n_chars
print "text preview:", raw_text[:500]

length of text: 141266
text preview: alices adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0




chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, and what is the use of a book, thought alice without pictures or
conversations?

so she was considering in her own mind as well as she could, for the
hot day mad


In [3]:
# write your code here

# extract all unique characters in the text
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print "number of unique characters found:", n_vocab

# create mapping of characters to integers and back
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# test our mapping
print 'a', "- maps to ->", char_to_int["a"]
print 25, "- maps to ->", int_to_char[25]

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100 #HYPER PARAMETER

inputs = []
outputs = []

for i in range(0, n_chars - seq_length, 1):
    inputs.append(raw_text[i:i + seq_length])
    outputs.append(raw_text[i + seq_length])
    
n_sequences = len(inputs)
print "Total sequences: ", n_sequences

#shuffle input and output data
indeces = range(len(inputs))
random.shuffle(indeces)

inputs = [inputs[x] for x in indeces]
outputs = [outputs[x] for x in indeces]

print inputs[0], "-->", outputs[0]

number of unique characters found: 37
a - maps to -> 11
25 - maps to -> o
Total sequences:  141166
ed up both its paws in surprise. what! never heard of
uglifying! it exclaimed. you know what to beau --> t


In [4]:
# create two empty numpy array with the proper dimensions
X = np.zeros((n_sequences, seq_length, n_vocab), dtype=np.bool)
y = np.zeros((n_sequences, n_vocab), dtype=np.bool)

# iterate over the data and build up the X and y data sets
# by setting the appropriate indices to 1 in each one-hot vector
for i, example in enumerate(inputs):
    for t, char in enumerate(example):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[outputs[i]]] = 1
    
print 'X dims -->', X.shape
print 'y dims -->', y.shape

X dims --> (141166, 100, 37)
y dims --> (141166, 37)


In [5]:
# define the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=False, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.50)) #HYPER PARAMETER
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [6]:
filepath="a6-basic_LSTM.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [11]:
epochs = 50 #HYPER PARAMETER
prediction_length = 100

for iteration in range(epochs):
    
    print 'epoch:', iteration + 1, '/', epochs
    #HYPER PARAMETER
    model.fit(X, y, validation_split=0.2, batch_size=256, nb_epoch=1, callbacks=callbacks_list)
    
    # get random starting point for seed
    start_index = random.randint(0, len(raw_text) - seq_length - 1)
    # extract seed sequence from raw text
    seed = raw_text[start_index: start_index + seq_length]
    
    print '----- generating with seed:', seed
    
    for diversity in [0.5, 1.2]:
        generate(seed, prediction_length, diversity)

epoch: 1 / 50
Train on 112932 samples, validate on 28234 samples
Epoch 1/1
112932/112932 [==============================] - 453s - loss: 2.3343 - val_loss: 2.1685
----- generating with seed: alice was very glad to find her in such a pleasant temper, and thought
to herself that perhaps it wa
alice was very glad to find her in such a pleasant temper, and thought
to herself that perhaps it wall the hert the the vert cat and the doon the manly aoul the walide the hand the the tat ile the thi
alice was very glad to find her in such a pleasant temper, and thought
to herself that perhaps it wat fdrthas seog thers.t
bumi illed nepday weved tar.i
woso cherth havne moucm ceker, sand mole yonos,
epoch: 2 / 50
Train on 112932 samples, validate on 28234 samples
Epoch 1/1
112932/112932 [==============================] - 494s - loss: 2.1802 - val_loss: 2.0522
----- generating with seed: ing on within--a constant howling and sneezing, and every now and then
a great crash, as if a dish o
ing on within-

**Do not change this code, but run it before submitting your assignment to generate the results**

In [8]:
def sample(preds, temperature=1.0): #HYPER PARAMETER
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
def generate(sentence, sample_length=50, diversity=0.35):
    generated = sentence
    sys.stdout.write(generated)

    for i in range(sample_length):
        x = np.zeros((1, X.shape[1], X.shape[2]))
        for t, char in enumerate(sentence):
            x[0, t, char_to_int[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = int_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print

In [13]:
prediction_length = 500
seed = "this time alice waited patiently until it chose to speak again. in a minute or two the caterpillar t"
generate(seed, prediction_length, .50)

this time alice waited patiently until it chose to speak again. in a minute or two the caterpillar to she said to the queen, and the time the court of the with a bet on the say, she said to herself, and notssed to say it to she to the chowded to come, in a finde say this, she was things were thing some to see the blatsel of come or a very to see it was a rest and as and the court, and the stopes so ther, and said to the share was good notheres of her thing soon thing the store his hand, and all the she went on any oney of tried and done of the beat mo cut.

as see the door beginn a were all ha
